In [10]:
import findspark 
findspark.init('C:\Spark\spark-3.0.0-preview2-bin-hadoop2.7')

In [12]:
from pyspark import SparkContext
try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

In [4]:
sales = [
    (1, 1, '01', '2017', 500), # sera la minima
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 2, '01', '2017', 500),
    (1, 1, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (1, 2, '01', '2016', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (2, 3, '01', '2017', 500),
    (4, 3, '01', '2017', 500),
    (4, 3, '01', '2017', 500),
    (4, 3, '02', '2017', 500),
    (4, 3, '03', '2017', 500),    

]

stores = [
    (1 , 'address 1', -1, -1, '30002', 'Georgia'),
    (2 , 'address 2', -2, -2, '30003', 'Georgia'),
    (3 , 'address 2', -3, -3, '30004', 'Georgia'),
    (4 , 'address 2', -4, -4, '10119', 'New York')    
]

In [7]:
# Obtener la tienda que realizó menor cantidad de ventas en el estado de "Georgia" en todo el año 2017

In [6]:
salesRdd = sc.parallelize(sales)
storesRdd = sc.parallelize(stores)

In [8]:
# filtramos la informacion de 2017 de ventas y realizamos un cambio de clave
salesToJoin = salesRdd.filter(lambda x: x[3] == '2017')\
    .map(lambda x: (x[1], (x[1],x[4]))) # (K,V) K: id_tienda, V: (id_tienda, total_ventas_mensuales)

# consideramos solo tiendas de Georgia
storesToJoin = storesRdd.filter(lambda x: x[5] == 'Georgia')\
    .map(lambda x: (x[0], x[0]))

In [9]:
# realizamos el join de ambos data frames
# simplificamos el formato para evitar los reduce complejos
# sumarizamos cantidad de ventas por tienda
# buscamos el minimo con reduce
salesToJoin.join(storesToJoin)\
    .map(lambda x: (x[0], x[1][0][1]))\
    .reduceByKey(lambda x, y: x+y)\
    .reduce(lambda x, y: x if x[1] < y[1] else y)

(1, 500)